# Tutorial 2: Stacking Sats Challenge

### Challenge Overview

**Hosted on [Hypertrial.ai](https://www.hypertrial.ai/)**

---

**Watch the General Overview on Youtube:** 
[![YouTube](https://img.shields.io/badge/Watch%20on-YouTube-red?logo=youtube&logoColor=white)](https://youtu.be/6YEeQ91TrSw?si=2FvsTeKsn0MjwRd8)


**Watch the Quant Deep Dive on Youtube:**
[![YouTube](https://img.shields.io/badge/Watch%20on-YouTube-red?logo=youtube&logoColor=white)](https://youtu.be/jYNRP98W1HA?si=tSKwPSo4L5H7McjN)

### 🧩 What Are You Solving?

Generalize Dollar Cost Averaging (DCA) to maximize Bitcoin accumulation by allowing **dynamic daily purchases**, while still preserving DCA’s core benefit: **systematic, consistent buying**.

---

### ⚙️ How You’ll Solve It

Build a **feature-driven model** that maps `historical BTC price data` to daily buy weights.  

> Each weight represents a portion of a fixed budget (normalized to 1) distributed over a 4-year cycle.

---

### 🎯 Objective

Maximize your model’s **SPD percentile** across **three 4-year cycles** within the backtest period `2013–2024`.

> **Satoshis per Dollar (SPD)** measures BTC accumulation per dollar:
> $$
\text{spd} = \left(\frac{1}{\text{BTC/USD}}\right) \times 100{,}000{,}000
$$
> **SPD Percentile** compares your model to the best and worst possible strategies:
> $$
\text{spd\_pct} = \left(\frac{\text{your SPD} - \text{worst SPD}}{\text{best SPD} - \text{worst SPD}}\right) \times 100
$$
> *All budgets are normalized to 1. Slippage is out of scope.*

---

### ✅ Model Constraints

1. **Positive Daily Purchases**  
   Every day must have: $\text{allocation}_t \geq 1 \times 10^{-5}$

2. **Budget Completeness**  
   Total daily allocations per cycle must sum to 1

3. **No Forward-Looking Data**  
   Use only current and past data—no peeking into the future

4. **Outperform Baseline**  
   Your SPD percentile must beat uniform DCA on all three cycles

---

## 🏁 Evaluation Criteria

Among valid models, winners are selected based on:

- **Highest SPD Percentile** averaged across the three 4-year cycles.

---

## 📦 Deliverables

**Model Code:**  
Submit your implementation using the provided template from [**Tutorial 3**](https://github.com/hypertrial/stacking_sats_challenge/blob/main/tutorials/3.%20Strategy%20Development%20Template.ipynb) on [**Hypertrial.ai**](https://www.hypertrial.ai/)

---

# Mathematical Deep Dive (For the Quants)

Let $N$ be the number of days in one accumulation cycle (4 years). Our total investment budget is normalized to 1 and allocated over these $N$ days using a vector of daily weights $\mathbf{w} = (w_1, w_2, \dots, w_N)$.

Each weight must satisfy the following:

$$
w_i \geq \text{MIN\_WEIGHT} = 10^{-5},\quad 
\sum_{i=1}^N w_i = 1.
$$

This guarantees a strictly positive allocation every day, avoiding zero-buys while respecting the full budget constraint.

---

### Uniform DCA (Baseline)

Uniform Dollar Cost Averaging corresponds to equal allocation on each day:

$$
w_i^{\text{uniform}} = \frac{1}{N},\quad \forall i \in \{1,\dots,N\}.
$$

This forms the baseline that your model must outperform.

---

### Dynamic-DCA: Model Definition and Optimization Objective

The objective of this challenge is to build an optimal **data-driven model** that generates a valid allocation vector over the accumulation cycle and improves upon uniform DCA.

A valid model is a function:

$$
f(\text{features}) \mapsto \mathbf{w} \in \mathrm{int}\,\Delta^{N-1}
$$

where:
- **Features** are observable inputs derived from available BTC price data
- $\mathbf{w}$ is a vector of daily allocation weights
- The output $\mathbf{w}$ must satisfy the following constraints:

$$
\mathrm{int}\,\Delta^{N-1} = \left\{ \mathbf{w} \in \mathbb{R}^N \,\middle|\, w_i \geq 10^{-5},\; \sum_{i=1}^N w_i = 1 \right\}
$$

This is the (clipped) probability simplex, which ensures:

- **Strictly positive daily purchases**: $w_i \geq 10^{-5}$  
- **Full budget utilization**: $\sum_i w_i = 1$

The model must use **only current and past data**—no future information is allowed. This ensures the strategy is deployable in real time and avoids overfitting.

---

### Objective: Maximize Sats per Dollar (SPD)

The performance metric is **sats-per-dollar (SPD)**, defined as:

$$
\mathrm{SPD}(\mathbf{w}) = \sum_{i=1}^N w_i \cdot \left( \frac{1}{p_i} \right)
$$

Here:
- $p_i$ is the closing BTC price on day $i$
- $\frac{1}{p_i}$ gives BTC per dollar
- $w_i$ scales the sats acquired by that day's budget share

Your goal is to build a model that outputs weights $\mathbf{w}$ which maximize SPD:

$$
\max_{f} \;\mathrm{SPD}(f(\text{features}))
$$

That is, based on a set of daily features (BTC price, on-chain metrics), your model must output a valid weight vector $\mathbf{w}$ that determines the buying schedule.

> **Note: Why Maximizing SPD ≡ Maximizing SPD Percentile**
>
> The SPD percentile is calculated as:
>
> $$
\text{spd\_pct} = \left(\frac{\text{strategy\_spd} - \text{worst\_spd}}{\text{best\_spd} - \text{worst\_spd}}\right) \times 100
$$
>
> where:
> - `best_spd` = buying 100% at the cycle low  
> - `worst_spd` = buying 100% at the cycle high  
> - Both values are **fixed** for a given cycle
>
> Because the denominator is constant for each cycle, and your strategy only affects `strategy_spd`, **maximizing SPD will also maximize SPD percentile**.

### Not Just Optimization — A Valid Predictive Model

This challenge is **not** about cherry-picking ideal weights using future prices.

> You must build a model that generalizes from observable features to weight allocations **without access to future data**.

In short, you’re solving a constrained optimization problem **indirectly**, through a model that maps features to valid decisions in a deployable way.

---

### Summary

Your model should:
- Take in **observable features** (at or before time $i$)
- Produce **strictly positive, normalized daily weights**
- Operate in a **non-forward-looking** manner (does not leak future data)
- **Outperform uniform DCA** across **ALL** backtested cycles
- Be evaluated using **SPD** (or its percentile form)

This preserves the benefits of DCA—discipline, regularity, and emotional neutrality—while introducing dynamic buys to capture more sats per dollar.


---

## Deeper Dive: Feature-Driven Rules for Dynamic DCA

In this framework, a valid strategy (or model) is defined as a deterministic mapping:

$$
f: \mathcal{X} \rightarrow \mathrm{int}\,\Delta^{N-1}
$$

where:
- $\mathcal{X}$ is the space of observable features (derived from BTC price data)
- $\mathrm{int}\,\Delta^{N-1}$ is the clipped simplex of valid weight vectors

At each cycle $t$, the model receives a feature vector $\mathbf{x}^{(t)}$ and outputs a valid allocation vector:

$$
\mathbf{w}^{(t)} = f\left( \mathbf{x}^{(t)}; \theta \right)
$$

with the constraints:

$$
w_i^{(t)} \geq 10^{-5},\quad 
\sum_{i=1}^N w_i^{(t)} = 1
$$

### Formalizing models this way: 

1. **Avoids hindsight bias.**  
   Allocations are **functionally generated**, not manually selected after seeing prices.

2. **Supports generalization.**  
   By choosing a parameterized function class (e.g., linear model, decision tree, neural net), models can be validated and tuned properly on historical cycles.

3. **Allows reproducibility and comparison.**  
   All strategies live in the same feasible space. Performance differences reflect model and feature quality—not optimization shortcuts.

4. **Encourages structural insight.**  
   Successful models reveal patterns and structural dynamics in Bitcoin’s behavior that improve DCA performance beyond uniform allocation.


**But it does not enforce the important non-forward looking criteria.** 

### Template for Non-Forward Looking Models

This template builds on the formal definiton of a model to outline a recursive Bayesian approach for developing models that ensure valid allocations while strictly adhering to the non-forward looking constraint . In this framework, each update depends only on all features observed up to the current day and on the most recent allocation, thereby simplifying the state history.

- **Initialization:**  
  Begin with a uniform prior over $N$ days:
  $$
  \mathbf{w}^{(0)} := \left[\frac{1}{N}, \frac{1}{N}, \dots, \frac{1}{N}\right] \in \mathrm{int}\left(\Delta^{N-1}\right)
  $$

  The pro of uniform weights is that they sum to 1 by definition. That is no longer the case when we have dynamic weights that are being allocated in a non-forward looking manner. 

- **Iterative Process:**  
  For each day $i$, update the allocation based on the features $(X_1, X_2, \dots, X_i)$ observed up to that day and on the allocation from the previous day, ensuring that each updated weight vector resides in the interior of the probability simplex:

  $$
  \begin{aligned}
    f\left(X_1,\, \mathbf{w}^{(0)}\right) &\mapsto \mathbf{w}_1 \in \mathrm{int}\left(\Delta^{N-1}\right) \quad \text{(@ day 1)} \\
    f\left(X_1, X_2,\, \mathbf{w}_1\right) &\mapsto \mathbf{w}_2 \in \mathrm{int}\left(\Delta^{N-1}\right) \quad \text{(@ day 2)} \\
    &\ \vdots \\
    f\left(X_1, \ldots, X_N,\, \mathbf{w}_{N-1}\right) &\mapsto \mathbf{w}_N \in \mathrm{int}\left(\Delta^{N-1}\right) \quad \text{(@ day N)}
  \end{aligned}
  $$

- **Final Allocation:**  
  Define the final weight vector for the cycle as:
  $$
  \mathbf{w} = \mathbf{w}_N \in \mathrm{int}\left(\Delta^{N-1}\right)
  $$

#### **Key Properties Ensured by the Template**

- **Non-Forward Looking:**  
  Each update $f\left(X_1, \ldots, X_i,\, \mathbf{w}_{i-1}\right)$ leverages only the features up to the current day and the immediately preceding allocation, strictly avoiding any forward-looking information.

- **Dynamic Budget Management:**  
  By relying on the most recent weight $\mathbf{w}_{i-1}$, the model can efficiently re-assess the remaining budget and rebalance allocations as needed, even allowing for early termination if too much of the budget is allocated early in the process.

- **Budget Conservation:**  
  The allocation vector remains on the simplex throughout, always satisfying:
  $$
  \sum_{i=1}^N w_i = 1 \quad \text{and} \quad w_i \geq 10^{-5} \quad \forall i
  $$

- **Adaptability:**  
  The function $f$ can be implemented using various model classes (e.g., linear models, decision trees, or neural networks), providing a flexible and reproducible framework across different strategies.


We recommend using this template as a starting point to ensure your model is not forward looking. 

---

## Geometry of the Optimization

Because SPD is a **linear function** of the allocation vector $\mathbf{w}$, its unconstrained maximum over the **closed** simplex

$$
\Delta^{N-1} = \left\{ \mathbf{w} \in \mathbb{R}^N \;\middle|\; w_i \geq 0,\; \sum_{i=1}^N w_i = 1 \right\}
$$

occurs at a **vertex**—in practice, this means allocating 100% of the budget to the single day with the lowest price.

To prevent such corner (all-in) solutions, we constrain the weights to the **interior** of the simplex:

$$
\mathrm{int}\,\Delta^{N-1} = \left\{ \mathbf{w} \in \mathbb{R}^N \;\middle|\; w_i \geq 10^{-5},\; \sum_{i=1}^N w_i = 1 \right\}
$$

This introduces a necessary tradeoff:

- **Maximizing SPD** favors leaning into lower-priced days  
- **Maintaining DCA principles** enforces a more balanced schedule (closer to uniform)

### Why constrain to the clipped (open) simplex?

- **Prevents degenerate, overfit allocations.**  
  A linear objective pushes toward extremes—forcing $w_i \geq 10^{-5}$ ensures diversified daily purchases.

- **Preserves DCA’s discipline.**  
  A strictly positive allocation guarantees that every day receives some investment.

- **Maintains convexity.**  
  The feasible region is convex: any average of two valid solutions is still a valid solution. This improves robustness and interpretability of the optimization landscape.